In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import molmodmt as m3t
import simtk.openmm as mm
import simtk.unit as unit
import simtk.openmm.app as app
import numpy as np
from sys import stdout

/home/diego/Myopt/Miniconda/miniconda3/envs/UIBCDF_lab_dev/lib/python3.7/site-packages/yank/experiment.py:1161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """)


# Preparación del sistema

## Procesando el archivo pdb
Cargamos el pdb del sistema y arreglamos con PDBFixer los posibles defectos como carencia de residuos, terminales o átomos:

In [3]:
system_modeller = m3t.fix_pdb_structure('MisL_Phyre.pdb',output_form='openmm.Modeller')

### Minimizamos el sistema

In [4]:
system_modeller_minimized = m3t.energy_minimization(system_modeller)

Potential Energy before minimization: 1942911898897101.8 kJ/mol
Potential Energy after minimization: -24026.381957107616 kJ/mol


In [5]:
if False:
    m3t.view([system_modeller, system_modeller_minimized])

In [6]:
initial_positions = m3t.get(system_modeller_minimized, coordinates=True)
topology = m3t.convert(system_modeller_minimized,'openmm.Topology')

# Tiramos de un extremo aplicando restraints y constraints a parte de la molécula.

### Listas de átomos sobre las que aplicar restricciones

Aplicamos restricciones espaciales de tipo harmónico a todos los átomos del dominio barril intra-membrana:

In [7]:
list_atoms_positions_restrained = m3t.select(topology, 'resid 183 to 504')

In [8]:
if False:
    m3t.view(system_modeller_minimized, selection=list_atoms_positions_restrained)

Aplicamos restricciones a todas las distancias relativas entre todos los átomos CA y CB correspondientes al "nucleo" del dominio extracelular:

In [9]:
list_CAs_distances_constrained = m3t.select(topology, 'resid 2 to 180 and name CA')
list_CBs_distances_constrained = m3t.select(topology, 'resid 2 to 180 and name CB')

In [10]:
if False:
    view = m3t.view(system_modeller_minimized, selection=list_CAs_distances_constrained)
    view.clear()
    view.add_ball_and_stick()
    view

## Creando el sistema a simular

In [11]:
forcefield = app.ForceField('amber99sbildn.xml')

In [12]:
system = forcefield.createSystem(topology, constraints=app.HBonds)

### Añadiños las fuerzas externas de posición (restraints) e internas de distancias (constraints)

### Restricciones harmónicas de posición

In [13]:
harmonic_restraint_potential = "0.5*K*((x-xo)^2 + (y-yo)^2 + (z-zo)^2)"

In [14]:
harmonic_restraint_force = mm.CustomExternalForce(harmonic_restraint_potential)

In [15]:
harmonic_restraint_force.addGlobalParameter('K', 50.0 * unit.kilocalories_per_mole/unit.angstrom**2)
harmonic_restraint_force.addPerParticleParameter('xo')
harmonic_restraint_force.addPerParticleParameter('yo')
harmonic_restraint_force.addPerParticleParameter('zo')

2

In [16]:
for index in list_atoms_positions_restrained:
    parameters = initial_positions[index].value_in_unit_system(unit.md_unit_system)
    harmonic_restraint_force.addParticle(int(index), parameters)

In [17]:
system.addForce(harmonic_restraint_force)

5

### Restricciones harmónicas de distancia

In [18]:
# https://simtk.org/api_docs/openmm/api4_1/python/classsimtk_1_1openmm_1_1openmm_1_1HarmonicBondForce.html
# https://simtk.org/api_docs/openmm/api4_1/python/classsimtk_1_1openmm_1_1openmm_1_1CustomBondForce.html#_details
harmonic_bond_force = mm.HarmonicBondForce()

In [19]:
num_atoms_list_CAs = len(list_CAs_distances_constrained)

k_cas = 10.0 * unit.kilocalories_per_mole/unit.angstrom**2

for ii in range(num_atoms_list_CAs):
    for jj in range(ii+1,num_atoms_list_CAs):
        #dist_ii_jj = m3t.distances(system_pdbfixer,ii,jj)
        vect = initial_positions[ii]-initial_positions[jj]
        dist_ii_jj = np.sqrt(vect[0]**2+vect[1]**2+vect[2]**2)
        harmonic_bond_force.addBond(int(ii), int(jj), dist_ii_jj, k_cas)

In [20]:
system.addForce(harmonic_bond_force)

6

### Tirando del extremo con fuerza externa

Otra manera más cómoda de añadir fuerzas es recurrir a la librería `openmmtools`:    
https://openmmtools.readthedocs.io/en/latest/gettingstarted.html#forces    
https://openmmtools.readthedocs.io/en/latest/forces.html    
https://openmmtools.readthedocs.io/en/latest/api/generated/openmmtools.forces.HarmonicRestraintBondForce.html#openmmtools.forces.HarmonicRestraintBondForce    

```python
from openmmtools.forces import HarmonicRestraintBondForce

num_atoms_list_CAs = len(list_CAs_distances_constrained)

k_cas = 500.0 * unit.kilocalories_per_mole/unit.angstrom**2

for ii in range(num_atoms_list_CAs):
    for jj in range(ii+1,num_atoms_list_CAs):
        distance_restraint = HarmonicRestraintBondForce (spring_constant=k_cas,
                                                         restrained_atom_index1=int(ii),
                                                         restrained_atom_index2=int(jj))
        system.addForce(distance_restraint)
```

## Creando la simulación

In [21]:
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 0*unit.kelvin
pressure    = None

In [22]:
friction   = 1000.0/unit.picosecond
step_size  = 2.0*unit.femtoseconds
integrator = mm.LangevinIntegrator(temperature, friction, step_size)
integrator.setConstraintTolerance(0.00001)

In [23]:
simulation_time = 0.05*unit.nanoseconds
saving_time = 0.005*unit.nanoseconds
simulation_steps = round(simulation_time/step_size)
saving_steps     = round(saving_time/step_size)
num_steps_saved  = round(simulation_steps/saving_steps)

In [24]:
print('Simulation steps:', simulation_steps)
print('Saving steps:', saving_steps)
print('Saved steps:', num_steps_saved)

Simulation steps: 25000
Saving steps: 2500
Saved steps: 10


In [25]:
from molmodmt.utils.openmm import check_platforms
check_platforms()

Plataforma Reference con velocidad 1.0
Plataforma CPU con velocidad 10.0
Plataforma CUDA con velocidad 100.0
Plataforma OpenCL con velocidad 50.0


In [26]:
platform = mm.Platform.getPlatformByName('CUDA')
properties = {'CudaPrecision': 'mixed'}

In [27]:
simulation = app.Simulation(topology, system, integrator, platform, properties)

In [28]:
simulation.context.setPositions(initial_positions)
simulation.context.setVelocitiesToTemperature(0*unit.kelvin)

In [29]:
simulation.minimizeEnergy()

In [30]:
simulation.reporters.append(app.DCDReporter('trajectory1.dcd', saving_steps))
simulation.reporters.append(app.StateDataReporter(stdout, saving_steps, step=True,
                            progress=True, remainingTime=True,
                            speed=True, totalSteps=simulation_steps, separator='\t'))

In [31]:
simulation.step(simulation_steps)

#"Progress (%)"	"Step"	"Speed (ns/day)"	"Time Remaining"
10.0%	2500	0	--
20.0%	5000	77.3	0:44
30.0%	7500	78.3	0:38
40.0%	10000	77.2	0:33
50.0%	12500	76.1	0:28
60.0%	15000	76.6	0:22
70.0%	17500	77.2	0:16
80.0%	20000	77.2	0:11
90.0%	22500	75.9	0:05
100.0%	25000	74.2	0:00


In [32]:
traj_md = m3t.load('trajectory1.dcd','mdtraj',topology=topology)

In [33]:
m3t.view(traj_md)

NGLWidget(count=10)

Esto podía haber sido hecho de manera más económica haciendo uso de por ejemplo https://openmmtools.readthedocs.io/en/latest/forces.html
o https://openmmtools.readthedocs.io/en/latest/_modules/openmmtools/forcefactories.html

https://github.com/uibcdf/Academia/blob/master/Din%C3%A1mica_Molecular/Pozo_Harm%C3%B3nico.ipynb
https://github.com/uibcdf/Academia/blob/master/Din%C3%A1mica_Molecular/MetEncefalina.ipynb
https://openmmtools.readthedocs.io/en/latest/_modules/openmmtools/forcefactories.html
http://www.maccallumlab.org/news/2015/1/23/testing
http://mdtraj.org/latest/examples/openmm.html
http://mdtraj.org/development/api/reporters.html
https://openmmtools.readthedocs.io/en/latest/gettingstarted.html